In [57]:
import dask
import dask.dataframe as dd
import dask.array as da
import dask.bag as bag
import h5py
import binascii

f = h5py.File('/media/dal/Localvore Volume/data.h5')
print(list(f.keys()))
ingr = f['ingrs_train', 'ingrs_test', 'ingrs_val']

interim = da.from_array(ingr, chunks=(1000, 20))
df = dd.from_dask_array(interim)
df.head()

['classes_test', 'classes_train', 'classes_val', 'ids_test', 'ids_train', 'ids_val', 'ilens_test', 'ilens_train', 'ilens_val', 'imnames_test', 'imnames_train', 'imnames_val', 'impos_test', 'impos_train', 'impos_val', 'ims_test', 'ims_train', 'ims_val', 'ingrs_test', 'ingrs_train', 'ingrs_val', 'numims_test', 'numims_train', 'numims_val', 'rbps_test', 'rbps_train', 'rbps_val', 'rlens_test', 'rlens_train', 'rlens_val', 'stvecs_test', 'stvecs_train', 'stvecs_val']


AttributeError: 'tuple' object has no attribute 'encode'

In [ ]:
import json
import dask.bag as bag
import dask.dataframe as dd
import ijson
from tqdm import tqdm
"""
dict_bag = bag.read_text('/media/dal/Localvore Volume/layer1.json',
                         blocksize=int(2**28)
                         ).map(json.loads)
print(dict_bag.count())
df = dict_bag.to_dataframe(columns=['id', 'title', 
                                    'instructions',
                                    'partition',
                                'url'])
df = df.repartition(npartitions=10)
"""
import time

timeout = 30
start = time.time()
while time.time() < start + timeout:
    for prefix, type, value in tqdm(ijson.parse(open('/media/dal/Localvore Volume/layer1.json'))):
        if prefix == 'item.title':
            print(value)


7131491it [01:02, 97666.43it/s]

Worlds Best Mac and Cheese
Dilly Macaroni Salad Recipe
Gazpacho
Crunchy Onion Potato Bake
Cool 'n Easy Creamy Watermelon Pie
Easy Tropical Beef Skillet
Kombu Tea Grilled Chicken Thigh
Strawberry Rhubarb Dump Cake
Yogurt Parfaits
Zucchini Nut Bread
Salmon & Salad a La SPORTZ
Fennel-Rubbed Pork Tenderloin with Roasted Fennel Wedges
Pink Sangria
Pineapple Loaf
Brown Sugar 'Karintou' Snacks
Corn Casserole
Grandmommy's Mexicali Meatloaf
Broccoli Rice Bake
Steak & Asparagus Wraps
Lentils Vegetable Soup
Harrison Muesli
Tinklee's Vanilla Crack
Erin's Mashed Potatoes
Banana Cream Pie Smoothie
Rice Paper Wrapped Halibut Roasted with Foie Gras Gingered Spaghetti Squash and Wild Mushroom Ragout
Frosty Lemon Ice Cream Sandwiches
Cooked Raspberry Jelly - CERTO Crystals
Apple Cinnamon French Toast Strata
Quick & Easy Crab Dip
Sausage Bread
Leek, Potato, and Bacon Casserole
Margarita Madras
Lemon Cupcakes with Blueberry Compote Filling and Cream Cheese Frosting
Soft Pretzels with Queso Poblano Sauce a

In [45]:
lines = [40, 35, 16475, 1143]
with open('/media/dal/Localvore Volume/vocab.txt') as f:
    for i, line in enumerate(f):
        if i in lines:
            print(line)

from

medium

marshmallows

chances



In [24]:
with open('/media/dal/Localvore Volume/vocab.bin', 'rb') as f:
    print(f.readline().decode())

30167 300



In [34]:
import word2vec
import sys

'''
Usage: python get_vocab.py /path/to/vocab.bin
'''
w2v_file = '/media/dal/Localvore Volume/vocab.bin'
model = word2vec.load(w2v_file)

vocab =  model.vocab

print("Writing to files/vocab.txt...")
f = open('/media/dal/Localvore Volume/vocab.txt','w')
f.write("\n".join(vocab))
f.close()

Writing to files/vocab.txt...


In [48]:
import pickle
with open('/media/dal/Localvore Volume/ingr_vocab.pkl', 'rb') as f:
    e = pickle.load(f)
    print(len(e))


1488


In [49]:
import pandas as pd
df = pd.read_json('http://data.csail.mit.edu/im2recipe/det_ingrs.json')
print(df.head())

           id                                        ingredients  \
0  000018c8a5  [{'text': 'penne'}, {'text': 'cheese sauce'}, ...   
1  000033e39b  [{'text': 'elbow macaroni'}, {'text': 'America...   
2  000035f7ed  [{'text': 'tomatoes'}, {'text': 'kosher salt'}...   
3  00003a70b1  [{'text': 'milk'}, {'text': 'water'}, {'text':...   
4  00004320bb  [{'text': 'watermelon gelatin'}, {'text': 'boi...   

                                               valid  
0  [True, True, True, True, True, True, True, Tru...  
1  [True, True, True, True, True, True, True, Tru...  
2  [True, True, True, True, True, True, True, Tru...  
3         [True, True, True, True, True, True, True]  
4                     [True, True, True, True, True]  


In [ ]:
import itertools
def compression(row):
    return list(itertools.compress(row['ingredients'], row['valid']))

#df['cleaned'] = df.apply(compression, axis=1)
df.drop(['cleaned', 'valid'], axis=1, inplace=True)
#df['ingredients'] = df['cleaned'].apply(lambda row: [item['text'] for item in row])
df.head()
df.to_json('/media/dal/Localvore Volume/ingr_dump.json',
           orient='records', lines=True)